In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

In [5]:
# data_dir = "/home/coder/early-exit/measurement_data/rpi"
data_dir = "measurement_data/rpi"
model_names = ["mobilenetv2", "resnet18", "resnet50"]

In [19]:
def remove_outliers_percentile_indices(data, lower_percentile=5, upper_percentile=95):
    # remove abnormal samples which are beyond the lower limit and upper limit
    lower_limit = np.percentile(data, lower_percentile)
    upper_limit = np.percentile(data, upper_percentile)

    # filtered_data = [value for value in data if lower_limit <= value <= upper_limit]
    abnormal_indices = [i for i, value in enumerate(data) if value < lower_limit or value > upper_limit]

    return abnormal_indices

def remove_outliers_percentile_data(data, lower_percentile=5, upper_percentile=95):
    # remove abnormal samples which are beyond the lower limit and upper limit
    lower_limit = np.percentile(data, lower_percentile)
    upper_limit = np.percentile(data, upper_percentile)

    filtered_data = [value for value in data if lower_limit <= value <= upper_limit]

    return filtered_data

# FLOPs

In [6]:
for model in model_names:
    df = pd.read_csv(os.path.join(data_dir, "flops", f"{model}.csv"))
    print(f"{model}:")
    print(df)

mobilenetv2:
   block1  block2    block3  block4
0  1.237M  1.223M  623.296K  1.764M
resnet18:
     block1    block2    block3    block4
0  154.145M  134.915M  134.579M  134.387M
resnet50:
     block1    block2    block3    block4
0  226.182M  339.773M  481.101M  265.081M


# Processing Time

## Models

In [12]:
# collect 100 samples for each group and calculate the average
# all data in ms
for model in model_names:
    df = pd.read_csv(os.path.join(data_dir, "proc_time", f"{model}_100.csv"))
    block1_data = df['block1']
    block2_data = df['block2']
    block3_data = df['block3']
    block4_data = df['block4']
    indices1 = remove_outliers_percentile_indices(block1_data)
    indices2 = remove_outliers_percentile_indices(block2_data)
    indices3 = remove_outliers_percentile_indices(block3_data)
    indices4 = remove_outliers_percentile_indices(block4_data)
    abnormal_indices = set(indices1) | set(indices2) | set(indices3) | set(indices4)

    filtered_df = df.drop(index=abnormal_indices)
    means = df.mean()
    print(f"{model}:")
    print(means)

mobilenetv2:
block1     7.653282
block2     9.336412
block3    12.624931
block4    11.492512
dtype: float64
resnet18:
block1    23.104622
block2    21.135542
block3    21.887453
block4    44.296980
dtype: float64
resnet50:
block1     65.567231
block2     68.984671
block3     85.544496
block4    104.341204
dtype: float64


## DQL

In [22]:
df = pd.read_csv(os.path.join(data_dir, "dql", "proc_time.csv"))
data = df['dql_time'].values
filtered_data = remove_outliers_percentile_data(data)
mean_data = sum(filtered_data) / len(filtered_data)
print(f"DQL processing time:")
print(f"{mean_data} ms")

DQL processing time:
16.47187868754069 ms


# Power Consumption

## Model Inference

In [25]:
# models are run every 0.1 second
df = pd.read_csv(os.path.join(data_dir, "VAs_meter", f"VAs_idle.csv"))
current = df['Current (A)'].values
voltage = df['Voltage (V)'].values
current_filtered = remove_outliers_percentile_data(current)
voltage_filtered = remove_outliers_percentile_data(voltage)

current_mean = sum(current_filtered)/len(current_filtered)
voltage_mean = sum(voltage_filtered)/len(voltage_filtered)
print("-----------")
print(f"idle:")
print(f"Voltage: {voltage_mean} V")
print(f"Current: {current_mean} A")

for model in model_names:
    df = pd.read_csv(os.path.join(data_dir, "VAs_meter", f"{model}_10_100.csv"))
    current = df['Current (A)'].values
    voltage = df['Voltage (V)'].values
    current_filtered = remove_outliers_percentile_data(current)
    voltage_filtered = remove_outliers_percentile_data(voltage)

    current_mean = sum(current_filtered)/len(current_filtered)
    voltage_mean = sum(voltage_filtered)/len(voltage_filtered)
    print("-----------")
    print(f"{model}:")
    print(f"Voltage: {voltage_mean} V")
    print(f"Current: {current_mean} A")

-----------
idle:
Voltage: 5.254138297872339 V
Current: 0.32381333333333323 A
-----------
mobilenetv2:
Voltage: 5.2259462365591425 V
Current: 0.48686555555555566 A
-----------
resnet18:
Voltage: 5.185109890109889 V
Current: 0.6899010989010991 A
-----------
resnet50:
Voltage: 5.170127659574468 V
Current: 0.8253744444444445 A


## Data transmission

In [26]:
# image size is 3*32*32, so the data size for transmission is 3*32*32*4=12288bytes≈12KB. Assume we transmit every 1s, the data transmission rate should be 12kbps
df = pd.read_csv(os.path.join(data_dir, "VAs_trans_meter", f"transmission_12kbps.csv"))
current = df['Current (A)'].values
voltage = df['Voltage (V)'].values
current_filtered = remove_outliers_percentile_data(current)
voltage_filtered = remove_outliers_percentile_data(voltage)
current_mean = sum(current_filtered)/len(current_filtered)
voltage_mean = sum(voltage_filtered)/len(voltage_filtered)
print("Data transmission power consumption:")
print(f"Voltage: {voltage_mean} V")
print(f"Current: {current_mean} A")

Data transmission power consumption:
Voltage: 5.251894736842107 V
Current: 0.34297340425531914 A


## DQL

In [27]:
## DQL is run every 0.1 second
df = pd.read_csv(os.path.join(data_dir, "dql", f"power.csv"))
current = df['Current (A)'].values
voltage = df['Voltage (V)'].values
current_filtered = remove_outliers_percentile_data(current)
voltage_filtered = remove_outliers_percentile_data(voltage)
current_mean = sum(current_filtered)/len(current_filtered)
voltage_mean = sum(voltage_filtered)/len(voltage_filtered)
print("DQL power consumption:")
print(f"Voltage: {voltage_mean} V")
print(f"Current: {current_mean} A")

DQL power consumption:
Voltage: 5.23165853658537 V
Current: 0.4587773584905662 A
